<a href="https://colab.research.google.com/github/kaniska/covid-19-hackathon/blob/master/social-network-mining/MedicalEntityRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#library imports

In [0]:
import tweepy
from tweepy import Stream
from tweepy import StreamListener 
from tweepy import OAuthHandler
import json

In [0]:
consumer_key = ""
consumer_secret = ""
access_token = ""
access_secret = ""

In [0]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

In [0]:
api = tweepy.API(auth)

In [0]:
@classmethod
def parse(cls, api, raw):
    status = cls.first_parse(api, raw)
    setattr(status, 'json', json.dumps(raw))
    return status

In [0]:
PREFIX = '/content/drive/My Drive/Colab Notebooks/datasets/covid-19-twitter-hack/'

In [0]:
# Status() is the data model for a tweet
tweepy.models.Status.first_parse = tweepy.models.Status.parse
tweepy.models.Status.parse = parse

class MyListener(StreamListener):
 
    def on_data(self, data):
        try:
            with open(PREFIX + 'tweet_stream.json', 'a') as f:
                f.write(data)
                return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
 
    def on_error(self, status):
        print(status)
        return True
 

In [0]:
#Set the hashtag to be searched

search_phrases= ['#covid','Drive through testing' , 'N95', 'respirator' , 'Ventilator' , 'sanitizer' , 'surgical face mask' , 'face masks' ]
twitter_stream = Stream(auth, MyListener())
twitter_stream.filter(track=search_phrases)

In [0]:
#view contents of json
data = open(PREFIX + 'tweet_stream.json').readlines()

In [0]:
len(data)

Analyze data to see geo data richness of tweets

In [0]:
def get_geo_data_richness(data):
  #Create dictionary to later be stored as JSON. All data will be included
  # in the list 'data'
  users_with_geodata = {
      "data": []
      }
  all_users = []
  total_tweets = 0
  geo_tweets  = 0

  for t in data:
    tweet =json.loads(t)
    if tweet['user']['id']:
      total_tweets+=1
      user_id = tweet['user']['id']

      if user_id not in all_users:
        all_users.append(user_id)

        #Give users some data to find them by. User_id listed separately 
        # to make iterating this data later easier

        user_data = {
                    "user_id" : tweet['user']['id_str'],
                    "features" : {
                        "name" : tweet['user']['name'],
                        "id": tweet['user']['id'],
                        "screen_name": tweet['user']['screen_name'],
                        "tweets" : 1,
                        "location": tweet['user']['location'],
                    }
                }
        #Iterate through different types of geodata to get the variable primary_geo
        if tweet['coordinates']:
          user_data["features"]["primary_geo"] = str(tweet['coordinates']['coordinates'][0]) + ", " + str(tweet['coordinates']['coordinates'][1])
          user_data["features"]["geo_type"] = "Tweet coordinates"
        elif tweet['place']:
          user_data["features"]["primary_geo"] = tweet['place']['full_name'] + ", " + tweet['place']['country']
          user_data["features"]["geo_type"] = "Tweet place"
        else:
          user_data["features"]["primary_geo"] = tweet['user']['location']
          user_data["features"]["geo_type"] = "User location"
        
        #Add only tweets with some geo data to .json
        if user_data["features"]["primary_geo"]:
          users_with_geodata['data'].append(user_data)
          geo_tweets += 1
          total_tweets +=1
    
    #If user already listed, increase their tweet count
    elif user_id in all_users:
      for user in users_with_geodata["data"]:
        if user_id == user["user_id"]:
          user["features"]["tweets"] += 1

  #Count the total amount of tweets for those users that had geodata            
  for user in users_with_geodata["data"]:
      geo_tweets = geo_tweets + user["features"]["tweets"]



  #Get some aggregated numbers on the data
  print("The file included " + str(len(all_users)) + " unique users who tweeted with or without geo data")
  print("The file included " + str(len(users_with_geodata['data'])) + " unique users who tweeted with geo data, including 'location'")
  print("The users with geo data tweeted " + str(geo_tweets) + " out of the total " + str(total_tweets) + " of tweets.")
  
  # Save data to JSON file
  with open(PREFIX + 'tweet_users_geo.json', 'w') as fout:
    fout.write(json.dumps(users_with_geodata, indent=4))



In [0]:
get_geo_data_richness(data)

## Todo
- Use openrefine to convert tweet_users_geo.json to csv
- Upload csv to Google Fusion Tables and tag location column for display.

In [0]:
!git status

In [0]:
!git add social-network-mining/MedicalFacilitiesDataCollection.ipynb

In [0]:
!git commit -m 'Crawl tweets based on medical kits & medical facilities terms'

In [0]:
!git push origin davies/medical-facilities-geodata